In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 0
setup_one_gpu(GPU)

Picking GPU 0


In [6]:
import tensorflow as tf
import numpy as np
import os.path as osp

from tflearn.layers.core import fully_connected
from tflearn.layers.conv import conv_2d

from tf_lab.point_clouds.encoders_decoders import decoder_with_fc_only
from tf_lab.external.structural_pc_losses import losses
from general_tools.notebook.tf import reset_tf_graph
from tf_lab.data_sets.numpy_dataset import NumpyDataset
from geo_tool import Point_Cloud

In [7]:
import helper

from tf_lab.diff_maps.in_out import raw_data, produce_net_data,\
prep_splits_labels_for_task, produce_diff_maps, classes_of_tasks

from tf_lab.diff_maps.basic_nets import pc_net, diff_mlp_net, diff_conv_net, pc_versions
from tf_lab.diff_maps.basic_nets import Basic_Net

In [33]:
diff_conv_net

In [30]:
%matplotlib inline

In [16]:
top_mesh_dir = '/orions4-zfs/projects/optas/DATA/Meshes/SCAPE_8_poses_2/'
gt_param_f = osp.join(top_mesh_dir, 'gt_shape_params.mat')
n_pose_classes = helper.n_pose_classes

In [17]:
use_pc = False
knn = 0
arch = 'conv'
sub_member_per_class = 50
n_shapes = sub_member_per_class * n_pose_classes
val_per = 0.10
test_per = 0.15
train_per = 1.0 - (val_per + test_per)
n_pc_points = 1024
task = 'regression'
mean_norm_diffs = True

In [21]:
top_data_dir = '/orions4-zfs/projects/optas/DATA/OUT/latent_diff_maps/experiments/SCAPE_8_poses_2'
synced_bases_file = osp.join(top_data_dir, '50_extract_%d_knn_50_fmapd.mat' % (knn,) )
n_cons = 40
seed = 42

In [22]:
gt_latent_params, in_pcs, pose_labels = \
raw_data(top_mesh_dir, gt_param_f, sub_member_per_class, n_pc_points)

n_classes = classes_of_tasks(task)

diff_maps = produce_diff_maps(synced_bases_file, n_cons, n_shapes)

splits, labels = \
prep_splits_labels_for_task(task, gt_latent_params, pose_labels, train_per, test_per, seed)

net_data = produce_net_data(in_pcs, splits, labels, diff_maps, use_pc, mean_norm_diffs)

#TODO mixed data.

In [31]:
nn_distance, approx_match, match_cost = losses()

In [10]:
# Try with     n_cons = 40
def diff_reconstructor(n_cons, n_pc_points):
    with tf.variable_scope('conv_based_reconstructor'):
        feed_pl = tf.placeholder(tf.float32, shape=(None, n_cons, n_cons))
        labels_pl = tf.placeholder(tf.float32, shape=(None, n_pc_points, 3))
        layer = tf.expand_dims(feed_pl, -1)
        layer = conv_2d(layer, nb_filter=10, filter_size=3, strides=2, activation='relu')        
        layer = conv_2d(layer, nb_filter=10, filter_size=4, strides=2, activation='relu')
        net_out = decoder_with_fc_only(layer, layer_sizes=[128, n_pc_points*3], b_norm=False)
    return net_out, feed_pl, labels_pl

In [ ]:
reset_tf_graph()
# sess = tf.InteractiveSession()

n_points = 2048
pc_dim = 3
n_classes = 2

x = tf.placeholder(tf.float32, [None, n_points, pc_dim])
y = tf.placeholder(tf.float32, [None, n_points, pc_dim])
l = tf.placeholder(tf.float32, [None, n_classes])


enc_filters = [64, 128, 128, 256]
split_of_embedding = 0.5
dim_thres = int(enc_filters[-1] * split_of_embedding)

mask_neurons = [100, 100, n_points]
decoder_neurons = [128, 256, n_points * 3]
discriminator_neurons = [128, 256]
gen_neurons = [128, 256]

# with tf.variable_scope('encoder') as scope:    
#     zx = encoder_with_convs_and_symmetry_new(x, n_filters=enc_filters, b_norm=False, scope=scope)
#     zx_sim = zx[:, :dim_thres]
#     zx_dissim = zx[:, dim_thres:]
    
#     zy = encoder_with_convs_and_symmetry_new(y, n_filters=enc_filters, b_norm=False, scope=scope, reuse=True)
#     zy_sim = zy[:, :dim_thres]
#     zy_dissim = zy[:, dim_thres:]
    
# with tf.variable_scope('mask_creator') as scope:
#     mask = decoder_with_fc_only(zx, layer_sizes=mask_neurons, b_norm=False, scope=scope)
#     mask_x_sim = tf.sigmoid(mask) >= 0.5    
#     mask_x_dissim = tf.logical_not(mask_x_sim)    
#     mask_x_sim = tf.tile(tf.expand_dims(mask_x_sim, 2), [1, 1, pc_dim])  # TODO d-check
#     mask_x_sim = tf.cast(mask_x_sim, tf.float32)    
#     mask_x_dissim = tf.tile(tf.expand_dims(mask_x_dissim, 2), [1, 1, pc_dim])  # TODO d-check
#     mask_x_dissim = tf.cast(mask_x_dissim, tf.float32)
        
#     mask = decoder_with_fc_only(zy, layer_sizes=mask_neurons, b_norm=False, scope=scope, reuse=True)
#     mask_y_sim = tf.sigmoid(mask) >= 0.5
#     mask_y_dissim = tf.logical_not(mask_y_sim)    
#     mask_y_sim = tf.tile(tf.expand_dims(mask_y_sim, 2), [1, 1, pc_dim])  # TODO d-check
#     mask_y_sim = tf.cast(mask_y_sim, tf.float32)    
#     mask_y_dissim = tf.tile(tf.expand_dims(mask_y_dissim, 2), [1, 1, pc_dim])  # TODO d-check
#     mask_y_dissim = tf.cast(mask_y_dissim, tf.float32)

# with tf.variable_scope('sim_decoder') as scope:
#     x_sim_pred = decoder_with_fc_only(zx_sim, layer_sizes=decoder_neurons, b_norm=False, scope=scope)
#     x_sim_pred = tf.reshape(x_sim_pred, [-1, n_points, pc_dim])
#     y_sim_pred = decoder_with_fc_only(zy_sim, layer_sizes=decoder_neurons, b_norm=False, scope=scope, reuse=True)
#     y_sim_pred = tf.reshape(y_sim_pred, [-1, n_points, pc_dim])

# with tf.variable_scope('dissim_decoder') as scope:    
#     x_dissim_pred = decoder_with_fc_only(zx_dissim, layer_sizes=decoder_neurons, b_norm=False, scope=scope)
#     x_dissim_pred = tf.reshape(x_dissim_pred, [-1, n_points, pc_dim])
#     y_dissim_pred = decoder_with_fc_only(zy_dissim, layer_sizes=decoder_neurons, b_norm=False, scope=scope, reuse=True)
#     y_dissim_pred = tf.reshape(y_dissim_pred, [-1, n_points, pc_dim])
    
# with tf.variable_scope('ae_losses') as scope:
#     x_sim_masked = tf.multiply(x, mask_x_sim)
#     x_dissim_masked = tf.multiply(x, mask_x_dissim)    
    
#     y_sim_masked = tf.multiply(y, mask_y_sim)
#     y_dissim_masked = tf.multiply(y, mask_y_dissim)
    
#     l1, _, s1, _ = nn_distance(x_sim_pred, y_sim_masked)
#     l2, _, s2, _ = nn_distance(x_dissim_pred, x_dissim_masked)
#     l3, _, s3, _ = nn_distance(y_sim_pred, x_sim_masked)
#     l4, _, s4, _ = nn_distance(y_dissim_pred, y_dissim_masked)    
#     ae_loss = tf.reduce_mean(l1 + l2 + l3 + l4)
#     ae_loss += tf.reduce_mean(s1 + s2 + s3 + s4)        

# with tf.variable_scope('generator') as scope:    
#     gen_in = tf.concat([zx_sim, zy_dissim, zx_dissim], axis=1)
#     gen_out = decoder_with_fc_only(gen_in, layer_sizes=gen_neurons + [dim_thres], b_norm=False, scope=scope)
#     gen_out = tf.nn.relu(gen_out)
#     gen_labels = tf.zeros_like(l)
    
# with tf.variable_scope('discriminator') as scope:
#     l_sizes = discriminator_neurons + [n_classes]
#     data_sim = tf.concat([zx_sim, zy_sim], axis=1)
#     data_logit = decoder_with_fc_only(data_sim, layer_sizes=l_sizes, b_norm=False, scope=scope)    
#     gen_sim = tf.concat([zx_sim, gen_out], axis=1)
#     gen_logit = decoder_with_fc_only(gen_sim, layer_sizes=l_sizes, b_norm=False, scope=scope, reuse=True)
    
# with tf.variable_scope('consistency_losses'):    
#     d_data_loss = tf.reduce_mean(tf.losses.log_loss(labels=l, predictions=tf.nn.softmax(data_logit)))
#     d_gen_loss = tf.reduce_mean(tf.losses.log_loss(labels=gen_labels, predictions=tf.nn.softmax(gen_logit)))
#     generator_push = -tf.reduce_mean(tf.losses.log_loss(labels=gen_labels, predictions=tf.nn.softmax(gen_logit)))